In [ ]:
!pip install -U tqdm music21

In [ ]:
!pip install -U google-api-python-client oauth2client PyDrive

In [ ]:
import numpy as np
from glob import glob
from tqdm import tqdm
from functools import reduce
from music21 import converter, instrument, note, chord, stream
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils.np_utils import to_categorical
from keras.utils import normalize
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, Bidirectional
from keras.models import Sequential, model_from_json
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import matplotlib.pyplot as plt
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D,RepeatVector
import keras
import os
import tensorflow as tf
from keras.layers import Input, Dense, MaxPooling1D, UpSampling1D
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard
from gensim.models import Word2Vec, KeyedVectors

Using TensorFlow backend.


## Práctica Guiada 2 - Regularización

Como hemos visto, uno de los problemas de los autoencoders es el overfitting. Cuando un autoencoder tiene una arquitectura con una gran cantidad de parámetros, corremos el riesgo de que aprenda perfectamente los datos de entrenamiento y pierda la capacidad de generalizar. 

En esta práctica vamos a regularizar nuestro modelo anterior LSTM.

Para observar la calidad de la generalización observamos el entrenamiento sobre conjuntos de train y test.

### Preprocesamos los datos


In [ ]:
def get_notes(audio_file):
    midi = converter.parse(audio_file)
    instrument_partition = instrument.partitionByInstrument(midi)
    audio_features = instrument_partition.parts[0].recurse()
    
    def get_note_chord(audio_feature):
        if isinstance(audio_feature, note.Note):
            return str(audio_feature.pitch)
        elif isinstance(audio_feature, chord.Chord):
            return '.'.join(map(lambda x: str(x), audio_feature.normalOrder))
    
    return list(filter(lambda x: x!= None, map(get_note_chord, audio_features)))

def get_chunks(file_notes):
    new_chunks = []
    num_chunks = len(file_notes) // 10
    for n in range(0,num_chunks):
        new_chunks.append(file_notes[n*10:n*10 + 10])
    return new_chunks

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


def download_from_drive(local_fn, id):
  print('Downloading: %s, id: %s' % (local_fn, id))
  downloaded = drive.CreateFile({'id': id})
  downloaded.GetContentFile(local_fn)

fns = [
    ('LSTMSinRegularizanTrTe.h5', '1dglMb7yK-tMocu5ufoyz3ofT5IeLzd5L'), 
    ('LSTMSinRegularizanTrTe.json', '1cI3-BiytVBsBAVISKwYdgCe_Zd6ArY7D'), 
    ('LSTMAutoencoder.h5', '1M1YIRGvhtDPWGKzwjldxTD_yrCWsaeWd'), 
    ('LSTMAutoencoder.json', '1SIZWIIim6wpJ4GW0Kqq2aVFajRVr0zFi'), 
    ('bach.tar.gz', '179tgDp-U3oPf0DuCIDuA59-L9nfhqBJN'), 
]

for local_fn, id in fns:
  download_from_drive(local_fn, id)

Downloading: LSTMSinRegularizanTrTe.h5, id: 1dglMb7yK-tMocu5ufoyz3ofT5IeLzd5L
Downloading: LSTMSinRegularizanTrTe.json, id: 1cI3-BiytVBsBAVISKwYdgCe_Zd6ArY7D
Downloading: LSTMAutoencoder.h5, id: 1M1YIRGvhtDPWGKzwjldxTD_yrCWsaeWd
Downloading: LSTMAutoencoder.json, id: 1SIZWIIim6wpJ4GW0Kqq2aVFajRVr0zFi
Downloading: bach.tar.gz, id: 179tgDp-U3oPf0DuCIDuA59-L9nfhqBJN


In [ ]:
!tar -xzf bach.tar.gz

In [ ]:
nmidis = 400
audio_files = sorted(glob("./bach/*.mid"), key=os.path.getsize)[0:nmidis]

In [ ]:
midis_notes_chords_by_file = []
for file in audio_files:
    try:
        notes_file = get_notes(file)
        midis_notes_chords_by_file.append(notes_file)
    except Exception as err:
        print(file, err)

./bach/458b2d957a32475619d157902a430866.mid 'NoneType' object has no attribute 'parts'
./bach/b92f0db2ea8e70515dddd1d73fbcfeb1.mid 'NoneType' object has no attribute 'parts'
./bach/60fc529f381b1ce1aaccc313215f9fb4.mid 'NoneType' object has no attribute 'parts'
./bach/19cf29a73f93f11c14972f647b3bdc4e.mid 'NoneType' object has no attribute 'parts'
./bach/24e3ce35ec2f4fcf2cd1adc296799810.mid 'NoneType' object has no attribute 'parts'
./bach/bc66a983c37d0dce783d90fa183ee739.mid 'NoneType' object has no attribute 'parts'
./bach/0f2a3840813cf4fe6e22dcf73666e12a.mid 'NoneType' object has no attribute 'parts'
./bach/327852b1e95de5032bb1db6f7738045f.mid 'NoneType' object has no attribute 'parts'
./bach/11c8c6bd558ce7512d454e9a395f1fb7.mid 'NoneType' object has no attribute 'parts'


In [ ]:
timesteps = 10

In [ ]:
all_chunks = []
for file in midis_notes_chords_by_file:
    all_chunks = all_chunks + get_chunks(file)

In [ ]:
# Pasamos a numpy Array
array_input = np.array(all_chunks)
# Instanciamos el LabelEncoder
le = LabelEncoder()
# El LabelEncoder "aprende" todas las clases que existen. 
# El método fit de LE exige un input unidimensional, por eso hacemos reshape.
le.fit(array_input.reshape(len(all_chunks)*timesteps))

# Usamos el diccionario aprendido por Label Encoder para transformar un array de dos dimensiones
notes_ids = le.transform(array_input.reshape(len(all_chunks)*timesteps)).reshape(len(all_chunks),timesteps)

Finalmente obtenemos el vector que utilizamos como input para la LSTM:

In [ ]:
# OneHotEncoding
one_hot_input_sequences = tf.squeeze(tf.one_hot(notes_ids, len(le.classes_))).numpy()

In [ ]:
one_hot_input_sequences.shape

(18335, 10, 256)

### Separación train/test

In [ ]:
np.random.seed(0)

In [ ]:
# Seleccionamos el 80% de los indices para train
train_index = np.random.randint(0,one_hot_input_sequences.shape[0],int(0.8 * one_hot_input_sequences.shape[0]))
X_train = one_hot_input_sequences[train_index,:,:]

In [ ]:
# Fancy indexing con la lista de test
test_index =  [x for x in range(one_hot_input_sequences.shape[0]) if x not in train_index]
X_test = one_hot_input_sequences[test_index,:,:]

## Implementación del modelo

In [ ]:
def auto_encoder_model(sequence_len, input_dim, latent_dim):
    # Encoder
    inputs = Input(shape=(sequence_len, input_dim))
    encoded = LSTM(input_dim*2, return_sequences=True)(inputs)
    encoded = LSTM(latent_dim)(encoded)
    # Decoder
    decoded = RepeatVector(sequence_len)(encoded)
    decoded = LSTM(input_dim*2, return_sequences=True)(decoded)
    decoded = Dense(input_dim, activation='softmax')(decoded)
    auto_encoder_model = Model(inputs, decoded)
    auto_encoder_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    auto_encoder_model.summary()
    return auto_encoder_model
    #encoder = Model(inputs, encoded)

In [ ]:
def save_model(model,name):
    model_json = model.to_json()
    with open("{}.json".format(name), "w") as json_file:
        json_file.write(model_json)
    model.save_weights("{}.h5".format(name))

In [ ]:
# Seteamos la reducción de dimensiones deseada
latent_dim = 5

In [ ]:
model = auto_encoder_model(timesteps, len(le.classes_), latent_dim)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 512)           1574912   
_________________________________________________________________
lstm_2 (LSTM)                (None, 5)                 10360     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 10, 5)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 512)           1060864   
_________________________________________________________________
dense_1 (Dense)              (None, 10, 256)           131328    
Total params: 2,777,464
Trainable params: 2,777,464
Non-trainable params: 0
_________________________________________________

In [ ]:
model.fit(X_train, X_train, 
          epochs=150, batch_size=128, shuffle=True,
          callbacks=[TensorBoard(log_dir='/tmp/autoencoder')],
          validation_data = (X_test, X_test))

Train on 14668 samples, validate on 8205 samples
Epoch 1/150
14668/14668 [==============================] - 12s 828us/step - loss: 4.0725 - val_loss: 3.8503
Epoch 2/150
14668/14668 [==============================] - 9s 603us/step - loss: 3.7744 - val_loss: 3.6924
Epoch 3/150
14668/14668 [==============================] - 9s 623us/step - loss: 3.6038 - val_loss: 3.5899
Epoch 4/150
14668/14668 [==============================] - 9s 620us/step - loss: 3.4861 - val_loss: 3.4957
Epoch 5/150
14668/14668 [==============================] - 9s 637us/step - loss: 3.4032 - val_loss: 3.4017
Epoch 6/150
14668/14668 [==============================] - 9s 620us/step - loss: 3.2653 - val_loss: 3.2166
Epoch 7/150
14668/14668 [==============================] - 9s 608us/step - loss: 3.1480 - val_loss: 3.1592
Epoch 8/150
14668/14668 [==============================] - 9s 614us/step - loss: 3.0724 - val_loss: 3.0711
Epoch 9/150
14668/14668 [==============================] - 9s 611us/step - loss: 2.9973 - val_

In [ ]:
save_model(model,'LSTMSinRegularizanTrTe')

In [ ]:
from keras.models import model_from_json

def load_model(fn_json, fn_h5):
    with open(fn_json) as f:
        model = model_from_json(f.read())
    model.load_weights(fn_h5)
    return model

model = load_model("LSTMSinRegularizanTrTe.json", "LSTMSinRegularizanTrTe.h5")

### Regularización L1 y L2

Ahora vamos a agregar un término de regularización L1 sobre la capa densa del modelo para intentar evitar que las curvas de loss de train y validation se separen tan abruptamente en unas pocas epochs. 

La fuerte separación entre ambas curvas demuestra que los parámetros del modelo están "aprendiendo" los datos de entrenamiento. 

Los términos de regularización permiten aplicar penalidades sobre los parámetros de las capas. Estas penalidades se incorporan a la función de loss que la red optimiza y sirven para restarle complejidad al modelo ajustado y así evitar un rápido overfitting. El término de penalización se calcula a partir de alguna función sobre los parámetros. La elección es agregar un término de penalización que sea una fracción (10e-5) de la norma l1 (la suma de los valores absolutos) y la norma l2 (la raíz cuadrada de la suma de los cuadrados) de los pesos de la red. 


In [ ]:
from keras import regularizers

In [ ]:
def auto_encoder_model_regularized(sequence_len, input_dim, latent_dim):
    # Encoder
    inputs = Input(shape=(sequence_len, input_dim))
    encoded = LSTM(input_dim*2, return_sequences=True)(inputs)
    encoded = LSTM(latent_dim)(encoded)
    # Decoder
    decoded = RepeatVector(sequence_len)(encoded)
    decoded = LSTM(input_dim*2, return_sequences=True)(decoded)
    decoded = Dense(len(le.classes_), activation='softmax',
                    activity_regularizer=regularizers.l2(10e-3))(decoded)
    auto_encoder_model = Model(inputs, decoded)
    auto_encoder_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    auto_encoder_model.summary()
    return auto_encoder_model
    #encoder = Model(inputs, encoded)

In [ ]:
model = auto_encoder_model_regularized(timesteps, len(le.classes_), latent_dim)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10, 256)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 512)           1574912   
_________________________________________________________________
lstm_5 (LSTM)                (None, 5)                 10360     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 10, 5)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 10, 512)           1060864   
_________________________________________________________________
dense_2 (Dense)              (None, 10, 256)           131328    
Total params: 2,777,464
Trainable params: 2,777,464
Non-trainable params: 0
_________________________________________________

In [ ]:
model.fit(X_train, X_train, 
          epochs=40, batch_size=128, shuffle=True,
          callbacks=[TensorBoard(log_dir='/tmp/autoencoder')],
            validation_data = (X_test, X_test))

Train on 14668 samples, validate on 8205 samples
Epoch 1/40
  512/14668 [>.............................] - ETA: 48s - loss: 5.4676 

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.222599). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


14668/14668 [==============================] - 11s 724us/step - loss: 4.3569 - val_loss: 4.1757
Epoch 2/40
14668/14668 [==============================] - 9s 591us/step - loss: 4.1087 - val_loss: 4.0458
Epoch 3/40
14668/14668 [==============================] - 9s 633us/step - loss: 3.9826 - val_loss: 3.9672
Epoch 4/40
14668/14668 [==============================] - 9s 620us/step - loss: 3.8741 - val_loss: 3.8516
Epoch 5/40
14668/14668 [==============================] - 9s 589us/step - loss: 3.7796 - val_loss: 3.7563
Epoch 6/40
14668/14668 [==============================] - 9s 597us/step - loss: 3.7105 - val_loss: 3.7197
Epoch 7/40
14668/14668 [==============================] - 9s 594us/step - loss: 3.6630 - val_loss: 3.6898
Epoch 8/40
14668/14668 [==============================] - 9s 610us/step - loss: 3.6322 - val_loss: 3.6685
Epoch 9/40
14668/14668 [==============================] - 9s 605us/step - loss: 3.6081 - val_loss: 3.6603
Epoch 10/40
14668/14668 [==============================]

Con la regularización observamos que si bien el loss es más elevado tanto en train como en test, la red puede entrenar durante más epochs sin experimentar un fuerte overfitting. 

In [ ]:
save_model(model,'LSTMRegularizado')

### Autoencoders con embeddings

Observamos que la regularización no es suficiente para generar buenas representaciones de los datos.
Otra posibilidad para implementar autoencoders es agregar una capa de embedding. 
Recordemos que los embeddings toman un vocabulario y mapean cada ítem de éste a un espacio vectorial de varias dimensiones. De esta manera, los embeddings reconocen distancias multidimensionales entre los elementos del vocabulario que resumen distintas formas de similitud que pueden aprenderse del corpus de entrenamiento.

En nuestro caso, los "documentos" son cada uno de los chunks de música de Bach y nuestro vocabulario son todas las posibles notas y acordes que figuran en los archivos midi. 

La función del embedding será tomar este input de 256 dimensiones y mapearlo en un espacio de secuencias.
Esto nos permite encontrar en nuestros datos, un equivalete musical de los sinónimos y por lo tanto codificar y decodificar la música con algunas variaciones. 


In [ ]:
def get_midi(note_chord_array, file_name):
    offset = 0
    output_notes = []
    # create note and chord objects based on the values generated by the model
    for pattern in note_chord_array:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.7
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(file_name))

In [ ]:
def lstm_embedding_model(sequence_len, input_dim, latent_dim):
    # Encoder
    inputs = Input(shape=(sequence_len, input_dim))
    encoded = Bidirectional(LSTM(latent_dim), merge_mode="sum")(inputs)
    # Decoder
    decoded = RepeatVector(sequence_len)(encoded)
    decoded = Bidirectional(LSTM(input_dim, return_sequences=True), merge_mode="sum")(decoded)
    auto_encoder_model = Model(inputs, decoded)
    auto_encoder_model.compile(optimizer="rmsprop", loss="mse")
    auto_encoder_model.summary()
    return auto_encoder_model
    #encoder = Model(inputs, encoded)

In [ ]:
def generate_embedding(all_chunks, embedding_size, embedding_file):
    all_chunks = list(map(lambda x: list(x), all_chunks))
    word2vec_model = Word2Vec(all_chunks, min_count=1, size=embedding_size)
    word2vec_model.save(embedding_file)
    return embedding_file

In [ ]:
def get_embedding(embedding_file):
    embedding_model = Word2Vec.load(embedding_file)
    print(embedding_model)
    return embedding_model

In [ ]:
def embedding_transform(embedding_model, all_chunks):
    return np.array(list(map(lambda chunk: embedding_model.wv[chunk], all_chunks)))

In [ ]:
def reverse_embedding(predict, embedding_model):
    return np.array(list(map(lambda vector: embedding_model.wv.similar_by_vector(vector, topn=1, restrict_vocab=None)[0][0], predict)))

In [ ]:
# Utilizamos los arrays originales que contienen chunks de Bach
array_input.shape

(18335, 10)

In [ ]:
embedding_size = 50
generate_embedding(array_input, embedding_size, 'model.bin')
embedding_model = get_embedding('model.bin')

Word2Vec(vocab=256, size=50, alpha=0.025)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
all_chunk_vectors = embedding_transform(embedding_model, array_input)

In [ ]:
all_chunk_vectors.shape

(18335, 10, 50)

In [ ]:
x_train, x_test = train_test_split(all_chunk_vectors, random_state=42, test_size = 0.2)

In [ ]:
input_dim = embedding_size
latent_dim = 25

In [ ]:
model = lstm_embedding_model(timesteps, input_dim, latent_dim)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 10, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 25)                15200     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 10, 25)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 50)            30400     
Total params: 45,600
Trainable params: 45,600
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, x_train, epochs=20, batch_size=128, shuffle=True, validation_data=(x_test, x_test), callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Train on 14668 samples, validate on 3667 samples
Epoch 1/20
  512/14668 [>.............................] - ETA: 1:03 - loss: 0.2216

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.286564). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


14668/14668 [==============================] - 12s 816us/step - loss: 0.1319 - val_loss: 0.1166
Epoch 2/20
14668/14668 [==============================] - 10s 672us/step - loss: 0.1120 - val_loss: 0.1101
Epoch 3/20
14668/14668 [==============================] - 10s 685us/step - loss: 0.1077 - val_loss: 0.1071
Epoch 4/20
14668/14668 [==============================] - 10s 698us/step - loss: 0.1060 - val_loss: 0.1061
Epoch 5/20
14668/14668 [==============================] - 10s 675us/step - loss: 0.1050 - val_loss: 0.1054
Epoch 6/20
14668/14668 [==============================] - 10s 706us/step - loss: 0.1043 - val_loss: 0.1050
Epoch 7/20
14668/14668 [==============================] - 10s 678us/step - loss: 0.1038 - val_loss: 0.1042
Epoch 8/20
14668/14668 [==============================] - 10s 703us/step - loss: 0.1034 - val_loss: 0.1039
Epoch 9/20
14668/14668 [==============================] - 10s 700us/step - loss: 0.1030 - val_loss: 0.1033
Epoch 10/20
14668/14668 [=======================

Escuchemos ahora uno de los audios originales y el output generado por el modelo.

In [ ]:
for i in range(30):
    prueba = x_train[i]
    muestra = reverse_embedding(prueba, embedding_model)
    print(muestra)
    get_midi(muestra, 'trainmuestra{}'.format(str(i)))
    predict = model.predict(prueba.reshape(1,10,50))[0]
    print(predict.shape)
    output = reverse_embedding(predict, embedding_model)
    print(output)
    get_midi(output, 'trainoutput{}'.format(str(i)))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['G#4' 'E3' 'E-4' 'C3' 'E4' 'C#3' 'C#4' 'E3' 'G#3' 'G#3']
(10, 50)
['G#4' 'G#4' '1.3.6' 'C#3' 'C#3' 'G#3' 'G#3' 'G#3' 'G#3' 'G#3']
['D4' 'G3' 'G5' 'D5' 'B-4' 'E-5' 'B-4' 'C4' 'F4' 'C5']
(10, 50)
['A3' '10.1.5' 'F5' 'F5' 'F5' 'C5' 'B-4' 'B-4' 'B-4' 'B-4']
['A4' 'F4' 'E4' 'F4' 'D4' 'C5' 'C4' 'A4' 'F3' 'B-4']
(10, 50)
['G4' 'G4' 'F4' 'F4' 'F4' 'F4' 'B-1' 'B-4' 'B-4' 'B-4']
['F4' 'C#3' 'C#4' 'G3' 'B-3' 'G4' '5.8' 'C4' '4.7' 'B-3']
(10, 50)
['E-4' 'E-4' 'B-1' 'B-1' 'B-1' '3' '3' '5.7' '5.7' '10.2']
['C5' 'E3' 'D3' 'B4' 'C3' 'C5' 'B2' 'D5' 'A2' 'C5']
(10, 50)
['C5' 'C3' 'D3' 'D3' 'D3' 'C3' 'C3' 'D5' 'D5' 'D5']
['3.9' 'G5' 'E4' 'B4' 'G4' 'A5' 'C4' 'F#4' 'D4' 'B5']
(10, 50)
['4.6.10' 'E-6' 'G6' 'B6' 'B6' 'G4' 'D4' 'D4' 'F6' 'B5']
['G3' 'E3' 'B3' 'E-4' 'C3' 'E4' 'G4' 'A2' 'F#4' 'E4']
(10, 50)
['G3' 'A3' 'A3' 'D4' 'E4' 'E4' 'E4' 'E4' 'E4' 'E4']
['F5' 'G5' 'A5' 'B5' 'C6' 'B5' 'C6' 'G4' 'F4' 'G4']
(10, 50)
['B-5' 'A5' 'A5' 'A5' 'A5' 'E6' 'E6' 'G6' 'G4' 'G4']
['B-4' 'F4' 'G#3' 'E-5' 'B-3' 'E-4' 'G3

In [ ]:
prueba = x_train[8]
muestra = reverse_embedding(prueba, embedding_model)
print(muestra)
get_midi(muestra, 'tramuestra8')

['B-4' 'F4' 'G#3' 'E-5' 'B-3' 'E-4' 'G3' 'E-4' 'C4' 'D4']


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
predict = model.predict(prueba.reshape(1,10,50))[0]
output = reverse_embedding(predict, embedding_model)
print(output)
get_midi(output, 'output8')

Lo interesante de usar un embedding es que podemos generar variaciones sobre el input original que suenan de forma similar pero son diferentes. 

In [ ]:
posibles_notas = np.unique(array_input)

In [ ]:
sequence_to_predict = np.random.choice(posibles_notas, size=10, replace=True)

Partiendo de una secuencia aleatorea, generamos una secuencia de musica

In [ ]:
print('############################')
long_sequence = []
range_ = 200
#sequence_to_predict = list(random_seq)


for i in range(range_):
    # Utilizo como input el embedding de los ultimos 10 valores de la secuencia
    network_input = embedding_transform(embedding_model,[sequence_to_predict])
    # Recibimos la transformacion
    new_music_embed = model.predict(network_input.reshape(1,10,50))[0]
    # Reversa del embedding
    new_music = list(reverse_embedding(new_music_embed, embedding_model))
    # Me guardo las ultimas notas de la secuencia generada
    
    long_sequence = long_sequence[:-8] + new_music
    # Cambio los dos ultimos valores de sequence_to_predict por un random
    sequence_to_predict = new_music
    random_song = np.random.choice(range(len(array_input)), size=1, replace=False)[0]
    
    # Avanzo la secuencia
    first_part =  sequence_to_predict[2:10]

    #first_part =  list(np.random.choice(posibles_notas, size=5, replace=False))
    last_part = list(array_input[random_song][0:2]) 
    sequence_to_predict = first_part + last_part
   
print(long_sequence)
get_midi(long_sequence[40:], 'GeneradaLSTM2')

############################


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['C5', '1.4.7.10', '0.3.5.8', 'G#1', '3.6.9', '3.6.9', 'B-1', 'B-1', 'C2', 'C2', 'C3', 'C3', 'C3', 'G2', 'G2', 'G2', '0.3.5.8', '0.3.5.8', '11.2.5.7', '1.4.8', '1.2', '1.2', 'C#4', '1.2', 'C#2', 'B-1', 'B-4', 'B-4', 'B-4', 'F4', 'F4', 'F4', 'B-1', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'G3', 'G3', 'A3', 'A3', 'A3', 'D4', 'D4', 'D4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'A4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A4', 'A4', 'G4', '1.4.7.10', '3.7.10', 'C#2', 'C#2', 'C#2', '1.2', 'E4', 'E4', 'E4', 'E4', 'B-1', 'B-1', 'D5', 'D5', 'D5', 'A4', 'A4', 'G4', 'G4', 'E4', 'E4', 'E4', 'A1', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'E5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', '11.1.4', '11.1.4', 'C#2', 'E2', 'F#2', 'F#2', 'F#2', 'F#2', '1.2', 'B1', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', '6.9.0.2', 'A3', 'A3', 'B3', 'B3', 'B3', 'B3', 'E4', 'E4', 'E4', 'B-1', 'C3', 'B-2', 'B-2', 'B-2', 'B-2', 'E-2', 'E-2', 'E-2', 'E-2', 'E-2', 'B-2', 'C3', 'C3', 'C3', 'C3'

In [ ]:
random_song = np.random.choice(range(len(array_input)), size=1, replace=False)[0]
sequence_to_predict = array_input[random_song]